<a href="https://colab.research.google.com/github/Suchit027/PCAP-Lab/blob/main/lab%205/q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://developer.download.nvidia.com/compute/cuda/11.8.0/local_installers/cuda_11.8.0_520.61.05_linux.run
!chmod +x cuda_11.8.0_520.61.05_linux.run
!./cuda_11.8.0_520.61.05_linux.run --silent --toolkit

--2025-02-01 18:22:27--  https://developer.download.nvidia.com/compute/cuda/11.8.0/local_installers/cuda_11.8.0_520.61.05_linux.run
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 23.52.40.16, 23.52.40.40
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|23.52.40.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4336730777 (4.0G) [application/octet-stream]
Saving to: ‘cuda_11.8.0_520.61.05_linux.run’

cuda_11.8.0_520.61. 100%[===================>]   4.04G   174MB/s    in 29s     

2025-02-01 18:22:57 (141 MB/s) - ‘cuda_11.8.0_520.61.05_linux.run’ saved [4336730777/4336730777]



In [15]:
%%writefile q2.cu
#include <stdio.h>
#include <stdlib.h>
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

__global__ void add(int *a, int *b, int *c, int threadlimit, int blocklimit){
  int t = threadIdx.x + 1;
  int bl = blockIdx.x + 1;
  int index = ((bl - 1) * threadlimit) + t - 1;
  if (bl <= blocklimit && t <= threadlimit){
    c[index] = a[index] + b[index];
  }
  return;
}
int main(){
  int *a, *b, *c;
  printf("enter n\n");
  int n;
  scanf("%d", &n);
  a = (int *)malloc(sizeof(int) * n);
  b = (int *)malloc(sizeof(int) * n);
  c = (int *)malloc(sizeof(int) * n);
  printf("enter array a\n");
  for(int i = 0; i < n; ++i){
    scanf("%d", &a[i]);
  }
  printf("enter array b\n");
  for(int i = 0; i < n; ++i){
    scanf("%d", &b[i]);
  }
  printf("enter threadlimit less than 256\n");
  int threadlimit;
  scanf("%d", &threadlimit);
  int blocklimit;
  if(n % threadlimit == 0){
    blocklimit = n / threadlimit;
  }
  else{
    blocklimit = (n / threadlimit) + 1;
  }
  int *da, *db, *dc;
  cudaMalloc((void **)&da, n * sizeof(int));
  cudaMalloc((void **)&db, n * sizeof(int));
  cudaMalloc((void **)&dc, n * sizeof(int));
  cudaMemcpy(da, a, n * sizeof(int), cudaMemcpyHostToDevice);
  cudaMemcpy(db, b, n * sizeof(int), cudaMemcpyHostToDevice);
  add<<<blocklimit, 256>>>(da, db, dc, threadlimit, blocklimit);
  cudaMemcpy(c, dc, n * sizeof(int), cudaMemcpyDeviceToHost);
  for(int i = 0; i < n; ++i){
    printf("%d ", c[i]);
  }
  cudaFree(da);
  cudaFree(db);
  cudaFree(dc);
  free(a);
  free(b);
  free(c);
  return 0;
}

Overwriting q2.cu


In [16]:
!nvcc q2.cu -o q2.out

In [18]:
!./q2.out

enter n
4
enter array a
1 2 3 4
enter array b
1 2 3 4
enter threadlimit less than 256
2
2 4 6 8 